In [1]:
%run -i ./startup.py
%run -i ./functions/dc_transform.py
%run -i ./functions/dc_join.py
%run -i ./functions/dc_list.py
%run -i ./functions/dc_keys.py
#%run -i ./functions/datagov_spider.py

In [2]:
import re
import nltk
import shapefile
import numpy as np
import seaborn as sns
import geopandas as gpd
import ipyleaflet as ipl

from branca.colormap import linear
from datetime import date
from unidecode import unidecode
from matplotlib import pyplot as plt

In [3]:
in_dir = '/home/josegois/data/maps/brazil/estados'
in_shp = in_dir + '/UFEBRASIL.shp'

br_estados = gpd.read_file(in_shp)

In [4]:
getl  = lambda df,i,v: df[df[i].isin(v)].reset_index(drop=True)
getin = lambda df,i,v: df[df[i].isin(np.unique(v))].reset_index(drop=True)
getre = lambda df,i,v: df[df[i].isin(list(filter(lambda i : re.findall(v,i),df[i])))].reset_index(drop=True)
dict_log = lambda d: {k:np.log10(v) for k,v in d.items()}

In [5]:
def dc_data(key_pattern='^.*$'):
    data = []
    for key in dc_join.dict.keys():
        if re.search(key_pattern,key):
            data.append(dc_join.dict[key]['data'])
    if ( len(data) == 1 ):
        return data[0]
    else:
        return data

In [6]:
def dc_notin(data, fields, exvals):
    n_ex       = np.empty((len(exvals),len(fields)))
    n_ex[:]    = 0

    for k,key in enumerate(fields):
        id_include    = np.empty((data.shape[0],),dtype=bool)
        id_include[:] = True

        for e,exval in enumerate(exvals):
            id_exval   = data[key].isin([exval]).values 
            n_ex[e,k]  = sum(id_exval) 
            id_include = id_include & ( ~id_exval )

        data = data.iloc[id_include]
    return (data,n_ex)

In [7]:
def levdist (A,B):
    n_a = len(A)
    n_b = len(B)

    states_distance = np.empty((n_a,n_b))

    for a in range ( n_a ):
        for b in range ( n_b ):
            states_distance[a,b] = nltk.edit_distance(A[a], B[b])
    return states_distance

In [8]:
dc_alias_re = {
    'discentes'                : '^dados_complementares_.*discentes$',
    'discentes_complementar'   : '.*socio_economicos_.*discentes*'
}

for dc_alias,dc_re in dc_alias_re.items():
    dc_join(db=db,dc=dc_re,alias=dc_alias)

/home/josegois/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [76]:
discentes = dc_data('^discentes$').merge(dc_data('^discentes_'),on='id_discente')

In [77]:
discentes = discentes.drop_duplicates(subset=['id_discente'])

In [78]:
discentes.estado_origem = discentes.estado_origem.astype('category')

In [79]:
discentes.id_discente

0         a2f469455a1226eca9d652d02ade7e4d
1         1cc536cdf93a5d423ca2f6da29c75eb5
2         ebe7eee3d3265baf1cb547335d58a1eb
5         62648116baf8df86606aa7b9c3be58ef
6         d8da593a8c6dec01a3853c2c54b6b9a6
                        ...               
213899    c751a8e17ba497115b334581a88ad7cc
213900    25e6a70b2e05066241d0b8fe708654f7
213902    f37d6ca2acd2a7790584c4b804c6db9b
213903    3f9ac03bb70877812be8c20a11b6df8d
213904    447ddef44cb303419f004a0b04b0e19d
Name: id_discente, Length: 63165, dtype: object

In [80]:
discentes.id_discente.unique().shape

(63165,)

In [81]:
(discentes,n_excluded) =  dc_notin(discentes, ['estado','estado_origem'], [np.nan, 'Ignorado/exterior','Não Informado'])

In [82]:
estado_decoder = lambda x: re.sub('[^a-z]','',unidecode(x.lower()))

br_estados['estado_decoded'] = br_estados.NM_ESTADO.apply(estado_decoder)
discentes['estado_decoded'] = discentes.estado.apply(estado_decoder)
discentes['estado_origem_decoded'] = discentes.estado_origem.apply(estado_decoder)

discentes = discentes.merge(br_estados[['CD_GEOCODU','estado_decoded']], left_on='estado_decoded', right_on='estado_decoded', suffixes=('','_1'))
discentes = discentes.merge(br_estados[['CD_GEOCODU','estado_decoded']], left_on='estado_origem_decoded', right_on='estado_decoded', suffixes=('','_2'))

In [83]:
br_geojson = {
        'type' : 'FeatureCollection',
        'features' : []
    }

for estado in br_estados.iloc:
    if 'Mul' in str(estado.geometry.__class__):
        coord = [list(map(lambda p: list(p), eg.exterior.coords)) for eg in estado.geometry]
    else:
        coord = [list(map(lambda p: list(p), estado.geometry.exterior.coords))]

    d = {
        'type' : 'Feature',
        'id' : estado.CD_GEOCODU,
        'properties' : {'name' : estado.NM_ESTADO},
        'geometry' : {
            'type' : 'Polygon',
            'coordinates' : coord
        }
    }
    br_geojson['features'].append(d)

In [84]:
dict_log = lambda d: {k:np.log10(v) for k,v in d.items()}

In [85]:
discentes_states = discentes.groupby(['CD_GEOCODU']).count()['id_discente']
layer_1_dict = discentes_states.to_dict()
layer_1_dict = dict_log(layer_1_dict)

In [86]:
br_layer_1 = ipl.Choropleth(
    geo_data=br_geojson,
    choro_data=layer_1_dict,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.9
    })

In [88]:
m = ipl.Map(center = (-20,-50), zoom = 3)
m.add_layer(br_layer_1)
m.add_control(ipl.FullScreenControl())
m

Map(center=[-20, -50], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [89]:
discentes_states_origin = discentes.groupby(['CD_GEOCODU_2']).count()['id_discente']
layer_2_dict = discentes_states_origin.to_dict()
layer_2_dict = dict_log(layer_2_dict)

In [90]:
br_layer_2 = ipl.Choropleth(
    geo_data=br_geojson,
    choro_data=layer_2_dict,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.9
    })

In [92]:
m2 = ipl.Map(center = (-20,-50), zoom = 3)
m2.add_layer(br_layer_2)
m2.add_control(ipl.FullScreenControl())
m2

Map(center=[-20, -50], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

NameError: name 'a' is not defined